In [1]:
!pip install azure-storage-blob # Microoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()


In [4]:
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

def get_week_of_year(date_str):
    """
    Calculate the ISO week number of the year for a given date.

    Parameters:
    date_str (str): A date string in the format 'YYYY-MM-DD'.

    Returns:
    int: ISO week number of the year.
    """
    # Parse the input string to a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    # Get the ISO calendar week number
    week_of_year = date.isocalendar()[1]

    return week_of_year

In [6]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'homework1'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)


homework311_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Display the head of the DataFrame
    print(df.shape)
    # sind I have only one csv, I am doing to do the following instructions
    homework311_df = df.copy()



homework1_311_20240420.csv
(100000, 18)


In [7]:
homework311_df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'incident_zip', 'street_name', 'city',
       'status', 'resolution_description', 'resolution_action_updated_date',
       'borough', 'open_data_channel_type', 'latitude', 'longitude',
       'location'],
      dtype='object')

In [8]:
#Consolidation
homework311_df['created_date']=pd.to_datetime(homework311_df['created_date'])
homework311_df['closed_date']=pd.to_datetime(homework311_df['closed_date'])
homework311_df['incident_zip'] =homework311_df['incident_zip'].astype('Int64')
homework311_df = homework311_df.drop(columns=['resolution_action_updated_date'])
homework311_df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,street_name,city,status,resolution_description,borough,open_data_channel_type,latitude,longitude,location
0,60903704,2024-04-19 01:58:13,NaT,DOT,Department of Transportation,Street Condition,Pothole,11211,METROPOLITAN AVENUE,BROOKLYN,Open,The Department of Transportation referred this...,BROOKLYN,UNKNOWN,NaN,NaN,NaN
1,60914765,2024-04-19 01:51:21,NaT,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,10013,MULBERRY STREET,NEW YORK,In Progress,NaN,MANHATTAN,ONLINE,40.714837,-73.999614,"\n, \n(40.71483730104454, -73.99961402373336)"
2,60914650,2024-04-19 01:51:19,NaT,NYPD,New York City Police Department,Blocked Driveway,Partial Access,11221,MONROE STREET,BROOKLYN,In Progress,NaN,BROOKLYN,MOBILE,40.688651,-73.922958,"\n, \n(40.68865115071951, -73.92295765130292)"
3,60906953,2024-04-19 01:50:56,NaT,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,11210,BEDFORD AVENUE,BROOKLYN,In Progress,NaN,BROOKLYN,ONLINE,40.636832,-73.953632,"\n, \n(40.63683244425841, -73.95363249985976)"
4,60908518,2024-04-19 01:50:49,NaT,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,10033,BROADWAY,NEW YORK,In Progress,NaN,MANHATTAN,MOBILE,40.852371,-73.934628,"\n, \n(40.85237108974576, -73.93462789776359)"


In [9]:
new_column_names = {
    'incident_zip': 'zip',
    'street_name': 'street'
}

homework311_df = homework311_df.rename(columns=new_column_names)
homework311_df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'zip', 'street', 'city', 'status',
       'resolution_description', 'borough', 'open_data_channel_type',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [10]:
#Cleaning
homework311_clean_df =homework311_df.dropna(subset=['closed_date'])
homework311_clean_df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,zip,street,city,status,resolution_description,borough,open_data_channel_type,latitude,longitude,location
19,60912336,2024-04-19 01:43:08,2024-04-19 01:50:51,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,11235,EAST 7 STREET,BROOKLYN,Closed,The Police Department responded to the complai...,BROOKLYN,ONLINE,40.591165,-73.962813,"\n, \n(40.59116515733906, -73.96281326934137)"
28,60909264,2024-04-19 01:39:24,2024-04-19 01:39:46,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10013,BROOME STREET,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,ONLINE,40.720080,-73.995804,"\n, \n(40.72007971694651, -73.9958044326403)"
47,60907762,2024-04-19 01:29:32,2024-04-19 01:45:16,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10009,EAST 10 STREET,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,PHONE,40.724570,-73.974143,"\n, \n(40.724570059316044, -73.97414302858385)"
52,60914003,2024-04-19 01:27:48,2024-04-19 01:46:32,NYPD,New York City Police Department,Noise - Residential,Loud Talking,10040,HILLSIDE AVENUE,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,ONLINE,40.858538,-73.929611,"\n, \n(40.8585382289902, -73.92961137673909)"
53,60902780,2024-04-19 01:27:28,2024-04-19 01:32:33,NYPD,New York City Police Department,Illegal Parking,Unauthorized Bus Layover,10002,MADISON STREET,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,ONLINE,40.712625,-73.990362,"\n, \n(40.712624620513665, -73.99036173504172)"


In [11]:
#Create date_diff, how many minutes

date_diff_minutes = []
for created_date, closed_date in zip(homework311_clean_df['created_date'], homework311_clean_df['closed_date']):
    date_diff = closed_date - created_date
    date_diff_minutes.append(int(date_diff.total_seconds() / 60))


homework311_clean_df['date_diff'] = date_diff_minutes

homework311_clean_df.tail()

<ipython-input-11-3ff83a6beb7c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homework311_clean_df['date_diff'] = date_diff_minutes


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,zip,street,city,status,resolution_description,borough,open_data_channel_type,latitude,longitude,location,date_diff
99994,60807978,2024-04-08 07:25:44,2024-04-08 08:43:01,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,11215,8 AVENUE,BROOKLYN,Closed,The Police Department responded to the complai...,BROOKLYN,MOBILE,40.666868,-73.978278,"\n, \n(40.66686766032023, -73.97827831511727)",77
99996,60811352,2024-04-08 07:25:03,2024-04-08 09:02:05,DSNY,Department of Sanitation,Vendor Enforcement,Food Vendor,10007,BROADWAY,NEW YORK,Closed,NaN,MANHATTAN,ONLINE,40.710256,-74.009620,"\n, \n(40.710255892218875, -74.00961988741942)",97
99997,60812597,2024-04-08 07:24:32,2024-04-08 19:17:17,NYPD,New York City Police Department,Illegal Parking,Paper License Plates,10467,ADEE AVENUE,BRONX,Closed,The Police Department responded to the complai...,BRONX,ONLINE,40.869680,-73.867812,"\n, \n(40.86967960735892, -73.86781162049007)",712
99998,60809091,2024-04-08 07:24:07,2024-04-08 11:39:11,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,11214,BAY 37 STREET,BROOKLYN,Closed,The Police Department issued a summons in resp...,BROOKLYN,MOBILE,40.596488,-73.991077,"\n, \n(40.59648750382108, -73.99107706612449)",255
99999,60810104,2024-04-08 07:24:07,2024-04-09 11:39:44,DSNY,Department of Sanitation,Dirty Condition,Trash,11218,EAST 3 STREET,BROOKLYN,Closed,The Department of Sanitation investigated this...,BROOKLYN,PHONE,40.638618,-73.976493,"\n, \n(40.63861801192124, -73.97649314983029)",1695


In [12]:
homework311_clean_df.shape

(72890, 18)

Create Dimension Tables

In [13]:
# Create Agency Dimension
agency_df = homework311_clean_df[['agency']].drop_duplicates()
agency_df['agency_id'] = range(1, len(agency_df) + 1)

new_order = ['agency_id', 'agency']
agency_dimension = agency_df[new_order]
agency_dimension.set_index('agency_id', inplace=True)
agency_dimension

,agency
agency_id,
1,NYPD
2,DSNY
3,DOT
4,DEP
5,HPD
6,DOB
7,DPR
8,DHS
9,TLC


In [14]:
agency_df = agency_df[['agency_id', 'agency']]
agency_df

,agency_id,agency
19,1,NYPD
424,2,DSNY
429,3,DOT
1016,4,DEP
1727,5,HPD
1858,6,DOB
1864,7,DPR
2267,8,DHS
3399,9,TLC
8974,10,DCWP


In [15]:
# Create Complaint Dimension
complaint_df = homework311_clean_df[['complaint_type']].drop_duplicates()
complaint_df['complaint_id'] = range(1, len(complaint_df) + 1)

complaint_dimension = complaint_df[['complaint_id','complaint_type']]
complaint_dimension.set_index('complaint_id', inplace=True)
complaint_dimension

,complaint_type
complaint_id,
1,Noise - Residential
2,Illegal Parking
3,Noise - Street/Sidewalk
4,Noise - Commercial
5,Noise - Vehicle
...,...
129,Bike Rack Condition
130,Borough Office
131,Indoor Sewage


In [16]:
# Create Channel Dimension
channel_df = homework311_clean_df[['open_data_channel_type']].drop_duplicates()
channel_df['channel_id'] = range(1, len(channel_df) + 1)

channel_dimension = channel_df[['channel_id','open_data_channel_type']]
channel_dimension.set_index('channel_id', inplace=True)
channel_dimension

,open_data_channel_type
channel_id,
1,ONLINE
2,PHONE
3,MOBILE
4,UNKNOWN
5,OTHER


In [25]:
# Create Date Dimension

start_date = pd.to_datetime('2024-04-08 00:00')
end_date = pd.to_datetime('2024-04-19 23:59')

date_range = pd.date_range(start=start_date, end=end_date, freq='T')
date_dimension = pd.DataFrame(date_range, columns=['date'])


date_dimension['year'] = date_dimension['date'].dt.year
date_dimension['month'] = date_dimension['date'].dt.month
date_dimension['day'] = date_dimension['date'].dt.day
date_dimension['hour'] = date_dimension['date'].dt.hour
date_dimension['minute'] = date_dimension['date'].dt.minute
date_dimension['date_iso_format'] = date_dimension['date'].apply(lambda x: x.isoformat())
date_dimension['date_id'] = (date_dimension['date'].dt.year * 100000000) + \
                            (date_dimension['date'].dt.month * 1000000) + \
                            (date_dimension['date'].dt.day * 10000) + \
                            (date_dimension['date'].dt.hour * 100) + \
                            date_dimension['date'].dt.minute

date_dimension = date_dimension[['date_id', 'date_iso_format','year','month','day','hour','minute']]
date_dimension

,date_id,date_iso_format,year,month,day,hour,minute
0,540617088,2024-04-08T00:00:00,2024,4,8,0,0
1,540617089,2024-04-08T00:01:00,2024,4,8,0,1
2,540617090,2024-04-08T00:02:00,2024,4,8,0,2
3,540617091,2024-04-08T00:03:00,2024,4,8,0,3
4,540617092,2024-04-08T00:04:00,2024,4,8,0,4
...,...,...,...,...,...,...,...
17275,540729443,2024-04-19T23:55:00,2024,4,19,23,55
17276,540729444,2024-04-19T23:56:00,2024,4,19,23,56
17277,540729445,2024-04-19T23:57:00,2024,4,19,23,57
17278,540729446,2024-04-19T23:58:00,2024,4,19,23,58


In [26]:
homework311_clean_df['created_date_id'] = (homework311_clean_df['created_date'].dt.year * 100000000) + \
                            (homework311_clean_df['created_date'].dt.month * 1000000) + \
                            (homework311_clean_df['created_date'].dt.day * 10000) + \
                            (homework311_clean_df['created_date'].dt.hour * 100) + \
                            homework311_clean_df['created_date'].dt.minute
homework311_clean_df['closed_date_id'] = (homework311_clean_df['closed_date'].dt.year * 100000000) + \
                            (homework311_clean_df['closed_date'].dt.month * 1000000) + \
                            (homework311_clean_df['closed_date'].dt.day * 10000) + \
                            (homework311_clean_df['closed_date'].dt.hour * 100) + \
                            homework311_clean_df['closed_date'].dt.minute
homework311_clean_df.head()

<ipython-input-26-da2d5b55f90b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homework311_clean_df['created_date_id'] = (homework311_clean_df['created_date'].dt.year * 100000000) + \
<ipython-input-26-da2d5b55f90b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homework311_clean_df['closed_date_id'] = (homework311_clean_df['closed_date'].dt.year * 100000000) + \


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,zip,street,city,status,resolution_description,borough,open_data_channel_type,latitude,longitude,location,date_diff,created_date_id,closed_date_id
19,60912336,2024-04-19 01:43:08,2024-04-19 01:50:51,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,11235,EAST 7 STREET,BROOKLYN,Closed,The Police Department responded to the complai...,BROOKLYN,ONLINE,40.591165,-73.962813,"\n, \n(40.59116515733906, -73.96281326934137)",7,540727231,540727238
28,60909264,2024-04-19 01:39:24,2024-04-19 01:39:46,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10013,BROOME STREET,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,ONLINE,40.720080,-73.995804,"\n, \n(40.72007971694651, -73.9958044326403)",0,540727227,540727227
47,60907762,2024-04-19 01:29:32,2024-04-19 01:45:16,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10009,EAST 10 STREET,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,PHONE,40.724570,-73.974143,"\n, \n(40.724570059316044, -73.97414302858385)",15,540727217,540727233
52,60914003,2024-04-19 01:27:48,2024-04-19 01:46:32,NYPD,New York City Police Department,Noise - Residential,Loud Talking,10040,HILLSIDE AVENUE,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,ONLINE,40.858538,-73.929611,"\n, \n(40.8585382289902, -73.92961137673909)",18,540727215,540727234
53,60902780,2024-04-19 01:27:28,2024-04-19 01:32:33,NYPD,New York City Police Department,Illegal Parking,Unauthorized Bus Layover,10002,MADISON STREET,NEW YORK,Closed,The Police Department responded to the complai...,MANHATTAN,ONLINE,40.712625,-73.990362,"\n, \n(40.712624620513665, -73.99036173504172)",5,540727215,540727220


In [19]:
# Create Location Dimension
location_df = homework311_clean_df[['zip','street','city','borough','latitude','longitude']].drop_duplicates()

location_df['location_id'] = range(1, len(location_df) + 1)

location_dimension = location_df[['location_id','city','borough','street','zip','latitude','longitude']]
location_dimension.set_index('location_id', inplace=True)
location_dimension

,city,borough,street,zip,latitude,longitude
location_id,,,,,,
1,BROOKLYN,BROOKLYN,EAST 7 STREET,11235,40.591165,-73.962813
2,NEW YORK,MANHATTAN,BROOME STREET,10013,40.720080,-73.995804
3,NEW YORK,MANHATTAN,EAST 10 STREET,10009,40.724570,-73.974143
4,NEW YORK,MANHATTAN,HILLSIDE AVENUE,10040,40.858538,-73.929611
5,NEW YORK,MANHATTAN,MADISON STREET,10002,40.712625,-73.990362
...,...,...,...,...,...,...
42179,BROOKLYN,BROOKLYN,BAY 40 STREET,11214,40.592881,-73.991247
42180,BROOKLYN,BROOKLYN,CORNELIA STREET,11221,40.691050,-73.914665
42181,BROOKLYN,BROOKLYN,AVENUE D,11236,40.652606,-73.903434


Create Fact Table

In [27]:
fact_table = homework311_clean_df[['created_date_id', 'closed_date_id', 'agency','complaint_type', 'city','borough', 'street','zip' ,'latitude', 'longitude', 'open_data_channel_type','date_diff']]
fact_table['fact_id'] = range(1, len(fact_table) + 1)
fact_table = fact_table[['fact_id','created_date_id', 'closed_date_id', 'agency','complaint_type', 'city','borough', 'street','zip' ,'latitude', 'longitude', 'open_data_channel_type','date_diff']]
fact_table = fact_table.merge(agency_df,on='agency',how='left')
fact_table = fact_table.drop(columns=['agency'])
fact_table = fact_table.merge(complaint_df,on='complaint_type',how='left')
fact_table = fact_table.drop(columns=['complaint_type'])
fact_table = fact_table.merge(channel_df,on='open_data_channel_type',how='left')
fact_table = fact_table.drop(columns=['open_data_channel_type'])
fact_table = fact_table.merge(location_df,on=['city','borough', 'street','zip' ,'latitude', 'longitude'],how='left')
fact_table = fact_table.drop(columns=['city','borough', 'street','zip' ,'latitude', 'longitude'])
fact_table


<ipython-input-27-720c4c4628fd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table['fact_id'] = range(1, len(fact_table) + 1)


,fact_id,created_date_id,closed_date_id,date_diff,agency_id,complaint_id,channel_id,location_id
0,1,540727231,540727238,7,1,1,1,1
1,2,540727227,540727227,0,1,1,1,2
2,3,540727217,540727233,15,1,1,2,3
3,4,540727215,540727234,18,1,1,1,4
4,5,540727215,540727220,5,1,2,1,5
...,...,...,...,...,...,...,...,...
72885,72886,540617813,540617931,77,1,2,3,2960
72886,72887,540617813,540617990,97,2,13,1,2984
72887,72888,540617812,540619005,712,1,2,1,36241
72888,72889,540617812,540618227,255,1,2,3,7249


Data Warehouse

In [21]:
# Specify the path to your JSON configuration file
config_file_path = 'config2.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

In [22]:
# Database connection URL
# Replace the placeholders with your actual database credentials
database_url = config["url"]
# Create a SQLAlchemy engine
engine = create_engine(database_url)

In [28]:
agency_dimension.to_sql('dim_agency', schema='request311', con=engine, if_exists='replace')
complaint_dimension.to_sql('dim_complaint', schema='request311', con=engine, if_exists='replace')
channel_dimension.to_sql('dim_channel', schema='request311', con=engine, if_exists='replace')
date_dimension.to_sql('dim_date', schema='request311', con=engine, if_exists='replace',index=False)
location_dimension.to_sql('dim_location', schema='request311', con=engine, if_exists='replace')

183

In [24]:
fact_table.to_sql('fact_request', schema='request311', con=engine, if_exists='replace',index=False)

890